# Защита модели от Adversarial примеров

Данная работа является продолжением предыдущей, в которой рассматривалось создание Adversarial примеров в задаче классификации изображений.

Атака оказалась крайне удачной, и в ходе данной работы будут рассмотрены способы это исправить

## Основные способы защиты, рассмотренные в работе:
- [Ensemble adversarial training](https://arxiv.org/pdf/1705.07204) - модификация adversarial training, в которой adv. examples генерируются несколькими другими моделями
- нормализация входных данных
- использование небольшого dropout

Для начала зададим все необходимые параметры

In [37]:
import numpy as np
import random
import torch
from pathlib import Path

device = "cuda" if torch.cuda.is_available() else "cpu"
root   = Path("datasets/svhn_cls")          # ваша готовая структура папок
BATCH  = 128
EPOCHS = 100
EPS    = 2/255                              # сила FGSM (L∞)
ADV_FRACTION = 0.5
SEED = 17
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
device

'cuda'

Для нормализации нам нужно посчитать среднее и дисперсию нашего датасета. Заметим, что они считаются только по обучающей выборке, чтобы тестовая выборка никак не влияла на процесс обучения

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from pathlib import Path

data_dir = Path("datasets/svhn_cls/train")

transform = transforms.ToTensor()
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

loader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=8)

mean = 0.
std = 0.
nb_samples = 0.

for data, _ in loader:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)  # (B, C, H*W)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

print("Mean:", mean)
print("Std:", std)

Mean: tensor([0.4377, 0.4438, 0.4728])
Std: tensor([0.1201, 0.1231, 0.1052])


Загрузим датасет, создадим трансформеры для него (с целью нормализации)

In [3]:
root = Path("datasets/svhn_cls")
mean = (0.4377, 0.4438, 0.4728)
std  = (0.1201, 0.1231, 0.1052)

tf_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
tf_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

ds_train = datasets.ImageFolder(root / "train", transform=tf_train)
ds_test  = datasets.ImageFolder(root / "test",  transform=tf_test)
dl_train = DataLoader(ds_train, batch_size=128, shuffle=True,  num_workers=4, pin_memory=True)
dl_test  = DataLoader(ds_test,  batch_size=256, shuffle=False, num_workers=4, pin_memory=True)

Создадим наш ансамбль моделей, которые будут генерировать adv. examples

Для этого возьмём готовый resnet и за finetune-им его на нашем датасете

In [8]:
import torch.nn.functional as F
from torchvision import models

def finetune_resnet(seed=0):
    torch.manual_seed(seed)
    net = models.resnet18(weights="IMAGENET1K_V1")
    net.fc = torch.nn.Linear(net.fc.in_features, 10)
    net.to(device)
    opt = torch.optim.Adam(net.parameters(), 1e-3)
    net.train()
    for _ in range(3):                      # быстрый 3-эпоховый fine-tune
        for x, y in DataLoader(ds_train, BATCH*2, shuffle=True):
            x, y = x.to(device), y.to(device)
            opt.zero_grad()
            F.cross_entropy(net(x), y).backward()
            opt.step()
    net.eval()
    for p in net.parameters():              # freeze
        p.requires_grad_(False)
    return net

src_nets = [finetune_resnet(seed) for seed in (42, 99)]

Создадим нашу основную модель, которую мы будем обучать

Последний (линейный) слой классификатора заменим, чтобы он предсказывал один из 10 классов (в предыдущей работе за нас это делал ultralytics, здесь же это приходится делать вручную)

Оптимизатор и планировщик так же раньше за нас добавляла библиотека, сейчас просто используем наиболее подходящие

In [18]:
from torch import nn
from ultralytics import YOLO

yolo = YOLO("yolov8s-cls.pt")
net  = yolo.model

head = net.model[-1]
in_f = head.linear.in_features
head.linear = nn.Linear(in_f, 10, bias=True)
net.to(device)

opt  = torch.optim.AdamW(net.parameters(), 1e-3)
sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, EPOCHS*len(dl_train))

Напишем свой вариант FGSM-атаки. Очень хотелось бы использовать реализацию из ART, однако она может работать только на процессоре, и постоянные переводы с GPU на CPU существенно замедлят обучение

In [26]:
def get_logits(model, x):
    out = model(x)
    return out[0] if isinstance(out, (tuple, list)) else out

def fgsm(images, labels, src_model):
    images = images.clone().detach().requires_grad_(True)
    logits = get_logits(src_model, images)
    loss    = F.cross_entropy(logits, labels)
    loss.backward()
    adv = (images + EPS * images.grad.sign()).clamp(0, 1).detach()
    return adv

Объявим ещё полезную функцию, которую будем вызывать каждые 10 эпох

In [29]:
def validate_and_checkpoint(
        epoch: int,
        net: torch.nn.Module,
        dl_test: torch.utils.data.DataLoader,
        ds_test_len: int,
        fgsm_fn,
        ckpt_dir: Path,
        opt: torch.optim.Optimizer,
        sched,
        best_robust: float,
        get_logits_fn,
        eps: float = 2 / 255
    ) -> float:
    net.eval()
    clean_hits = robust_hits = 0

    for x, y in dl_test:
        x, y = x.to(device), y.to(device)

        with torch.no_grad():
            clean_logits = get_logits_fn(net, x)

        adv_x = fgsm_fn(x, y, net)

        with torch.no_grad():
            adv_logits = get_logits_fn(net, adv_x)

        clean_hits  += (clean_logits.argmax(1) == y).sum().item()
        robust_hits += (adv_logits.argmax(1)  == y).sum().item()

    clean_acc  = clean_hits  / ds_test_len
    robust_acc = robust_hits / ds_test_len

    print(f"\nEpoch {epoch:02d}: "
          f"clean {clean_acc:.3f} | FGSM ε={eps} {robust_acc:.3f}")

    # ----------- сохранить обычный чекпоинт -----------
    ckpt_path = ckpt_dir / f"epoch_{epoch:02d}.pth"
    torch.save({
        "epoch": epoch,
        "model_state": net.state_dict(),
        "optimizer_state": opt.state_dict(),
        "scheduler_state": sched.state_dict(),
        "clean_acc": clean_acc,
        "robust_acc": robust_acc,
    }, ckpt_path)
    print(f"Saved checkpoint ➜ {ckpt_path.name}")

    # ----------- сохранить лучший ---------------------
    if robust_acc > best_robust:
        best_robust = robust_acc
        best_path   = ckpt_dir / "best.pth"
        torch.save(net.state_dict(), best_path)
        print(f"New best robust_acc = {best_robust:.3f} ➜ {best_path.name}")

    net.train()            # вернуться к обучению
    return best_robust

Наконец, можем переходить к обучению

In [38]:
from tqdm import tqdm

ckpt_dir = Path("checkpoints_eat")
ckpt_dir.mkdir(exist_ok=True, parents=True)

best_robust = 0.0

for epoch in range(1, EPOCHS + 1):
    net.train()
    running_acc = 0
    pbar = tqdm(dl_train, desc=f"Epoch {epoch:02d}/{EPOCHS}", leave=False)

    for step, (imgs, lbls) in enumerate(pbar, 1):
        imgs, lbls = imgs.to(device), lbls.to(device)

        # добавляем adv. examples
        src = random.choice(src_nets + [net])
        adv = fgsm(imgs, lbls, src)
        k   = int(imgs.size(0) * ADV_FRACTION)

        mix_imgs   = torch.cat([imgs[:k], adv[:k]])
        mix_labels = torch.cat([lbls[:k], lbls[:k]])

        #  шаг оптимизации
        opt.zero_grad()
        logits = get_logits(net, mix_imgs)
        loss   = F.cross_entropy(logits, mix_labels, label_smoothing=0.1)
        loss.backward(); opt.step(); sched.step()

        # обновляем прогресс бар
        pred = logits.argmax(1)
        running_acc += (pred == mix_labels).sum().item()
        avg_acc = running_acc / (step * mix_labels.size(0))
        pbar.set_postfix(loss=f"{loss.item():.3f}",
                 clean_acc=f"{(pred[:k]==lbls[:k]).float().mean():.3f}",
                 mix_acc=f"{avg_acc:.3f}")

    if epoch % 10 == 0 or epoch == EPOCHS:
        best_robust = validate_and_checkpoint(
            epoch         = epoch,
            net           = net,
            dl_test       = dl_test,
            ds_test_len   = len(ds_test),
            fgsm_fn       = fgsm,
            ckpt_dir      = ckpt_dir,
            opt           = opt,
            sched         = sched,
            best_robust   = best_robust,
            get_logits_fn = get_logits,
            eps           = EPS
        )


Epoch 10: clean 0.330 | FGSM ε=0.00784313725490196 0.334
Saved checkpoint ➜ epoch_10.pth
New best robust_acc = 0.334 ➜ best.pth



Epoch 20: clean 0.317 | FGSM ε=0.00784313725490196 0.323
Saved checkpoint ➜ epoch_20.pth



Epoch 30: clean 0.306 | FGSM ε=0.00784313725490196 0.307
Saved checkpoint ➜ epoch_30.pth



Epoch 40: clean 0.290 | FGSM ε=0.00784313725490196 0.299
Saved checkpoint ➜ epoch_40.pth



Epoch 50: clean 0.286 | FGSM ε=0.00784313725490196 0.298
Saved checkpoint ➜ epoch_50.pth



Epoch 60: clean 0.282 | FGSM ε=0.00784313725490196 0.295
Saved checkpoint ➜ epoch_60.pth


KeyboardInterrupt: 